In [ ]:
import matplotlib.pyplot 
import torch
from omegaconf import OmegaConf
from pytorch_lightning.callbacks import RichProgressBar

from data_n import *
from r_roberta import *

cfg = OmegaConf.load(f"/opt/ml/code/pl/config/r_roberta.yaml")

pl.seed_everything(cfg.train.seed, workers=True)

# dataloader와 model을 생성합니다.
dataloader = Dataloader(
    cfg.model.model_name,
    cfg.train.batch_size,
    cfg.data.shuffle,
    cfg.path.train_path,
    cfg.path.test_path,
    cfg.train.seed,
)
model = Model(cfg)

# gpu가 없으면 'gpus=0'을, gpu가 여러개면 'gpus=4'처럼 사용하실 gpu의 개수를 입력해주세요
trainer = pl.Trainer(
    precision=16,
    accelerator="gpu",
    devices=1,
    max_epochs=cfg.train.max_epoch,
    callbacks=[RichProgressBar()],
    auto_lr_find=True,
    # deterministic=True,
    # limit_train_batches=0.15,  # use only 30% of training data
    # limit_val_batches = 0.01, # use only 1% of val data
    # limit_train_batches=10    # use only 10 batches of training data
)

lr_finder = trainer.tuner.lr_find(model=model,datamodule=dataloader, num_training=300)
lr_finder.results

fig = lr_finder.plot(suggest=True)

new_lr = lr_finder.suggestion()
print('new_lr :',new_lr)

fig.show()